In [111]:
import pandas as pd
import requests
import json

# from web3 import Web3

In [149]:
factory_logs = pd.read_csv('/Users/andrewgoldberg/Projects/llamapay-analytics/data/etherscan/factory_eventlogs.csv')

In [162]:
factory_tokens = factory_logs['data'].apply(lambda x: f'0x{(x[26:66])}')
len(factory_tokens)

45

In [165]:
factory_tokens

45

In [164]:
factory_logs['topics'].iloc[0]

"['0x7d620f75be49cb3047050bea486a013fcdf9187edfb1afbef44fbd42585469ac']"

In [131]:
payerDeposit = '/Users/andrewgoldberg/Projects/llamapay-analytics/data/eventscanner/llamapay_events/llamapay_payerDeposit.json'
payerWithdraw = '/Users/andrewgoldberg/Projects/llamapay-analytics/data/eventscanner/llamapay_events/llamapay_payerWithdraw.json'
streamCancelled = '/Users/andrewgoldberg/Projects/llamapay-analytics/data/eventscanner/llamapay_events/llamapay_streamCancelled.json'
streamCreated = '/Users/andrewgoldberg/Projects/llamapay-analytics/data/eventscanner/llamapay_events/llamapay_streamCreated.json'
streamCreatedWithReason = '/Users/andrewgoldberg/Projects/llamapay-analytics/data/eventscanner/llamapay_events/llamapay_streamCreatedWithReason.json'
streamModifed = '/Users/andrewgoldberg/Projects/llamapay-analytics/data/eventscanner/llamapay_events/llamapay_streamModified.json'
streamPaused = '/Users/andrewgoldberg/Projects/llamapay-analytics/data/eventscanner/llamapay_events/llamapay_streamPaused.json'
withdraw = '/Users/andrewgoldberg/Projects/llamapay-analytics/data/eventscanner/llamapay_events/llamapay_withdraw.json'

with open(payerDeposit) as f:
    payerDepositJson = json.load(f)

with open(payerWithdraw) as f:
    payerWithdrawJson = json.load(f)

with open(streamCancelled) as f:
    streamCancelledJson = json.load(f)

with open(streamCreated) as f:
    streamCreatedJson = json.load(f)

with open(streamCreatedWithReason) as f:
    streamCreatedWithReasonJson = json.load(f)

with open(streamModifed) as f:
    streamModifiedJson = json.load(f)

with open(streamPaused) as f:
    streamPausedJson = json.load(f)

with open(withdraw) as f:
    withdrawJson = json.load(f)


In [132]:
def fromJsonToDataFrame(eventJson):
    df = pd.json_normalize(eventJson['blocks']).T.rename_axis('cols').reset_index()
    df['block'], df['transactionHash'], df['logIndex'], df['topics'] = df.cols.str.split('.', n=4).str
    df = df.rename(columns = {0:'values'})
    df = df.pivot(index=['block', 'transactionHash', 'logIndex'], columns=['topics'], values=['values'])
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    df = df.reset_index()
    return df

In [133]:

payerDeposit_df = fromJsonToDataFrame(payerDepositJson)
payerWithdraw_df = fromJsonToDataFrame(payerWithdrawJson)
streamCancelled_df = fromJsonToDataFrame(streamCancelledJson)
streamCreated_df = fromJsonToDataFrame(streamCreatedJson)
streamCreatedWithReason_df = fromJsonToDataFrame(streamCreatedWithReasonJson)
streamModified_df = fromJsonToDataFrame(streamModifiedJson)
streamPaused_df = fromJsonToDataFrame(streamPausedJson)
withdraw_df = fromJsonToDataFrame(withdrawJson)


/var/folders/c7/8ppmyxcn6znc4ql4nfmfksy80000gn/T/ipykernel_33436/499605929.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['block'], df['transactionHash'], df['logIndex'], df['topics'] = df.cols.str.split('.', n=4).str
/var/folders/c7/8ppmyxcn6znc4ql4nfmfksy80000gn/T/ipykernel_33436/499605929.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['block'], df['transactionHash'], df['logIndex'], df['topics'] = df.cols.str.split('.', n=4).str
/var/folders/c7/8ppmyxcn6znc4ql4nfmfksy80000gn/T/ipykernel_33436/499605929.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['block'], df['transactionHash'], df['logIndex'], df['topics'] = df.cols.str.split('.', n=4).str
/var/folders/c7/8ppmyxcn6znc4ql4nfmfksy80000gn/T/ipykernel_33436/499605929.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['block'], df

In [134]:
payerDeposit_df

,block,transactionHash,logIndex,values_amount,values_from,values_timestamp
0,14704397,0x1b08acc378b050baf5b9cd834c44ed2e71affc40c221...,60,60000000000,0x6BFeBA63E725bA2610Bf251CE43De50b4d18A944,2022-05-03T11:31:02
1,14832012,0xdaa9f651c47d2827a24463966493ec6a835765c624b3...,253,25952000000,0x61334C283bF42741E9682263F5b2a7271Df32Ed0,2022-05-23T21:52:19
2,14852397,0x428470e9a11286d78789bbbede7780cb26d8d442d218...,15,32000000000000000000000,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,2022-05-27T05:38:07
3,14878966,0x2f86d04ba42f206babd9ec71e70642426d1573a9f4a1...,450,75000000000,0x8a8dBE201A6688711539dC45F8033812a093b22b,2022-05-31T13:31:55
4,14908165,0x7979a77ab8a30bc6cd12e1df92e5ba0478a8907caf6e...,65,43333000000000000000000,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,2022-06-05T08:46:43
...,...,...,...,...,...,...
604,19152759,0x613f030269e664281b3f8fef82dc22d178e402472cc7...,572,756000000000000000000,0x98915686E23032a1D3b8596BaDD7fAdf147d5c00,2024-02-04T05:04:47
605,19160468,0xe3822f195a3e9c55b28000af73c3a78e9fb90916c983...,474,28000000000,0xDdc01A9FCB5eA881E596FFD8bdE4ee66C16adc99,2024-02-05T07:03:35
606,19166479,0x01fcf4425489de4d6ec061e0251e917ca9a5407bd017...,106,15000000000000000000,0xe6fB38CF3276b464C6Ec90dA78491c7e0E9d8353,2024-02-06T03:19:47
607,19166490,0x414982f07322dfc9d249fa34b5838d1bd5fff87a9e45...,119,138000000000000000000,0xe6fB38CF3276b464C6Ec90dA78491c7e0E9d8353,2024-02-06T03:21:59


In [ ]:
etherscan_api_key = 'RQCKJ2RUXUBASPGW1W4MM4HW7VDF9VKURJ'
llamapay_factory_address = '0xde1C04855c2828431ba637675B6929A684f84C7F'

etherscan_eventlogs = f'https://api.etherscan.io/api?module=logs&action=getLogs&address={llamapay_factory_address}&apikey={etherscan_api_key}'

request_factory_eventlogs = requests.get(etherscan_eventlogs)

In [135]:
payerWithdraw_df

,block,transactionHash,logIndex,values_amount,values_from,values_timestamp
0,14861309,0xdb09f2ffe6847773e938cdb0455912f17e4d46cb445c...,181,20000000000,0x61334C283bF42741E9682263F5b2a7271Df32Ed0,2022-05-28T16:24:04
1,14941460,0x679f028a6acb99fda2bc770d994efd6f4a8c17281a26...,366,22879420,0x05A8810bf69a6d9496497a364F1e21E855187f95,2022-06-11T00:50:38
2,15227526,0x78d34940f323cca79b71289952ed2aa6e609d61e5afc...,434,2000000000,0x8a8dBE201A6688711539dC45F8033812a093b22b,2022-07-27T23:35:11
3,15277414,0x45bb9b7dfda5d6d6e16696dd0f76e95c57f691ea8b50...,493,5000000000,0x8a8dBE201A6688711539dC45F8033812a093b22b,2022-08-04T18:18:54
4,15321926,0x7d1addf651ae88ede407ed47a4c011f4f950001d58c6...,476,114087301585669,0xCD298D9872823F0665b7726604dB286eD3376A8b,2022-08-11T17:10:08
...,...,...,...,...,...,...
72,18965097,0x35c207e4b8c5c9737f46bc147fa87896daa36a7cef0d...,281,999384920634920636,0x98915686E23032a1D3b8596BaDD7fAdf147d5c00,2024-01-08T21:55:35
73,19021264,0x6145d35dfc7111bd19a8cae6008c60c38c25445a77fa...,347,20000000000000000000000,0x572865DF00967098faC87Dfc7d297097652Ab510,2024-01-16T18:33:23
74,19090854,0xf14d2bae55606ae2bb98210146821a3347fa6a59b379...,77,100000000000000000000000,0xe9970f8985aC8bF672223dEd6545cC7A4c7C55d5,2024-01-26T12:43:59
75,19110665,0x8d8a9dd9449f1479c3070b791ccd6a9c1991d3eed0d8...,229,250000000000000000000000,0xa869fbceDf9D0D574957C728febDca802f5f61C7,2024-01-29T07:21:59


In [146]:
streamCancelled_df.iloc[0]['transactionHash']

'0x6bd1ac5d16d8c66076d90e957a650eef47be16d4ed7ff2a82893eaabf52dad46'

In [148]:
streamCancelled_df

,block,transactionHash,logIndex,values_amountPerSec,values_from,values_streamId,values_timestamp,values_to
0,14976801,0x6bd1ac5d16d8c66076d90e957a650eef47be16d4ed7f...,135,8267195767195767,0x945C08edBE84d734cb2CD2D1FFb78dD3246130ea,0x0de1735c43db2d2d6d688b709660ac9c6e2710c03cf5...,2022-06-17T02:48:34,0xFE5eE99FDbcCFAda674A3b85EF653b3CE4656e13
1,15018714,0xe397d5682ef780b5371f8c80670e0cd94b4f945c7b43...,361,578703703703703703,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x3d1677fe10b10f4741d8ea00d6a42d35afb35cbeaee5...,2022-06-24T14:24:09,0x8aC9D995AC53dc58d5b9BEef430077fC4Eac2035
2,15165339,0xe0c6d9d43a292e0c0a6d389b235a3440696bea3b6a2e...,200,52950767511632,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0xa43a09e5bfe7a206df88cb866b0382d364b4638e9ef1...,2022-07-18T07:40:04,0xB0fc0776cEd4A6C2e39c080279040a6EEA303B57
3,15165339,0xe0c6d9d43a292e0c0a6d389b235a3440696bea3b6a2e...,203,385802469135802432,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x5da686a8662e882d71d2e4f9ed56b3a59ccbd98beb33...,2022-07-18T07:40:04,0xB0fc0776cEd4A6C2e39c080279040a6EEA303B57
4,15173171,0x91656eb3246a560498e808cbfb00af0518ec12f50eff...,419,192901234567901234,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0xd634cf4ed24cbb7ce73d0764bcd0067c7d31f9143836...,2022-07-19T12:53:08,0x908dcDb61189B56F5Cb7B0c60d332E3eE18d9300
...,...,...,...,...,...,...,...,...
293,19057497,0xf753da358266d2eb09e4e7302b1ea9e7ee2844de46d6...,311,115740740740740741,0x5973955FF9a77B4BE34f7040fA965894Deec6767,0x8a68f16952fcb43cfad9c9e51476440233c0079056be...,2024-01-21T20:13:11,0x099fA51ADB19Ce28A74D3125493bA162AdfA3032
294,19066907,0xcb4855a7a1b70101da76ee4e0adc626ef7bd875997ed...,254,597993827160493827,0x387b1216b84E36c1e155cf18F1e763fBd6A8c4a9,0xaef85ce7fc5aa426dbaf9f4bcaf4de8beb3e7d2c13e1...,2024-01-23T04:09:11,0x9094219BDEBd7bABF74C593C347224dc261b36c2
295,19066989,0x2b04ed158df7c68391acf45eda9dc67b2ac576725958...,110,353665123456790123,0x387b1216b84E36c1e155cf18F1e763fBd6A8c4a9,0xc1d92350184cef8804c517c90ec838c10fda7a6417ba...,2024-01-23T04:25:47,0x338aC1Db72033B0b67d9514f41b8C68649D77e35
296,19101349,0xaf972a4c7206bef83f2b08ebc8b00a78064f4f38c21c...,115,208950617283950617,0xc3A1BDddbeC6Bc896162F22d0D7e260Df8fE01C5,0x312207d1e40dc7498380a05f7f66214c6488370ec79e...,2024-01-28T00:03:11,0xd90E9181B20D8D1B5034d9f5737804Da182039F6


In [137]:
streamCreated_df

,block,transactionHash,logIndex,values_amountPerSec,values_from,values_streamId,values_timestamp,values_to
0,15018714,0xe397d5682ef780b5371f8c80670e0cd94b4f945c7b43...,355,56423611111111,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x51af03ee7fc82ca8477e573c68a88941028d67cd2a36...,2022-06-24T14:24:09,0x8727FF875E070d73B678cCea85BF9FF01c05A785
1,15018714,0xe397d5682ef780b5371f8c80670e0cd94b4f945c7b43...,358,385802469135802469,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x2f58336103c65ec4bb32e2facb3211f06eef789bc84c...,2022-06-24T14:24:09,0xbd7B3Bc2C4581Fd173362c830AE45fB9506B3dA5
2,15018928,0xcfae5f001f0b248c115be3e626f8361eb3c71af3dd03...,133,514390432098765432,0x1434f19804789e494E271F9CeF8450e51790fcD2,0x2acffe4afb67b9380f0e4874331ffe6f013995a80f77...,2022-06-24T15:16:49,0x53954EcA473B695AC6c83A096B961557C0240cEa
3,15019133,0x547c4632d2d89497769029efdbc7b99bac7b5e0d866c...,485,458140432098765432,0x1434f19804789e494E271F9CeF8450e51790fcD2,0x18273c61365d6d5929a6c22690d140b63891736a8146...,2022-06-24T16:11:32,0x9eb9e3dc2543dc9FF4058e2A2DA43A855403F1fD
4,15022977,0x81573273a443667816233c6d58506558f21c353de892...,405,38580246913580,0x8D5b48934c0C408ADC25F14174c7307922F6Aa60,0xd44a32ce03bde5c5a232def52bd5817824b53d7a8636...,2022-06-25T09:16:08,0x80F4A80123A0b9e1f8303095bd50225638e6B13C
...,...,...,...,...,...,...,...,...
699,19136786,0x95209278f08ba77d8d9a71d2d52f08304ac2e6375796...,174,128761574074074,0xe6fB38CF3276b464C6Ec90dA78491c7e0E9d8353,0x2ee3d71ff5067bfd7af84ab91ff839c80e7e954235e8...,2024-02-01T23:13:35,0xA39B9fDE6df958d860fDeFb9a2b415feFd60B53C
700,19136786,0x95209278f08ba77d8d9a71d2d52f08304ac2e6375796...,175,128761574074074,0xe6fB38CF3276b464C6Ec90dA78491c7e0E9d8353,0x438bd55350aae387b77e73be6417b341717dc9507d88...,2024-02-01T23:13:35,0xaa89c1EeA3F5C9686b42e2baD20d8BeCA5d939F5
701,19136786,0x95209278f08ba77d8d9a71d2d52f08304ac2e6375796...,176,88734567901235,0xe6fB38CF3276b464C6Ec90dA78491c7e0E9d8353,0xad39d75c0447506d270a693ce9277f0ffb2fcfc642bc...,2024-02-01T23:13:35,0x49113E2fCf5F1f411ac59ECAA7f5Fc9509879D84
702,19137855,0xc6e398185045037a82a4d79454c06a66352dcbf954a2...,422,96450617283950617,0x98915686E23032a1D3b8596BaDD7fAdf147d5c00,0x5da33affc4e9426c0d5bb8d53f2bd7b5c980d90457fe...,2024-02-02T02:50:11,0x0D221aD79D29F2a85Be423F457b94d08e29E9924


In [138]:
streamCreatedWithReason_df

,block,transactionHash,logIndex,values_amountPerSec,values_from,values_reason,values_streamId,values_timestamp,values_to
0,15632680,0xa96246a18846cd6966fc87571313a2021d8e1e384661...,507,385802469135802469,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,hope,0x045b01834a24af74f35ec94a08e9dec99e07ed567520...,2022-09-28T15:17:47,0xe5Ed591e0d6A39aB14cd9a9B788EddBd49C4c259
1,15632680,0xa96246a18846cd6966fc87571313a2021d8e1e384661...,508,52950617283950,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,hope,0xaaae42d430b5868ca7fdd6864b0a6dd5ea2cb754e5f4...,2022-09-28T15:17:47,0xe5Ed591e0d6A39aB14cd9a9B788EddBd49C4c259
2,15632680,0xa96246a18846cd6966fc87571313a2021d8e1e384661...,509,158443692534057,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,yip-70,0x4925dadbab1f5000d2b5312cfb24402c8a9b7afa9fa0...,2022-09-28T15:17:47,0x187089b65520D2208aB93FB471C4970c29eAf929
3,15632680,0xa96246a18846cd6966fc87571313a2021d8e1e384661...,510,231481481481481481,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,gh-89,0xaeb5f06266fa413524728e2f963ca175b972f43850a3...,2022-09-28T15:17:47,0xDCaDB75dF0EB724C2cBd44ff322AF782C50Ee53B
4,15632680,0xa96246a18846cd6966fc87571313a2021d8e1e384661...,511,115740740740740740,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,gh-89,0xaf1d3b4db181a6eff938313e0588e0fcab7d24ac4201...,2022-09-28T15:17:47,0xF4606161D94445F66cA26E834b373C9316613FA8
5,15632680,0xa96246a18846cd6966fc87571313a2021d8e1e384661...,512,115740740740740740,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,gh-89,0x3601c453d61baae0c89aa3729386560b313152950d84...,2022-09-28T15:17:47,0xD3f2A11c58c88E9575f0964214b927A7Ea9e4B90
6,15632680,0xa96246a18846cd6966fc87571313a2021d8e1e384661...,513,115740740740740740,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,gh-89,0xfdd957e82aee29625fe72781b00dc46bfc237c925f30...,2022-09-28T15:17:47,0x714D8b5874b3a6a69f289f4e857F4C9670074296
7,15659839,0x57bc99f6007989606bdd9d1adf91c99d198de51f61d2...,81,192901234567901234,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,gh-92,0xbdd58f49765c047c4dabc4963512a449955683e31db2...,2022-10-02T10:23:35,0x114E6997a17D6964C1e42B575bc1e60ca2202CD5
8,15659839,0x57bc99f6007989606bdd9d1adf91c99d198de51f61d2...,82,655864197530864197,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,gh-93,0x6e558f30bdeb194e8314c09813946007e4264fc69e7b...,2022-10-02T10:23:35,0xBd5CA40C66226F53378AE06bc71784CAd6016087
9,15876854,0x08ce69eab43acbeae999314c5661d7452b52cfda4c44...,367,266362252663622526,0x79387c398982D8f3Ef0BBaf49c5e9E89D7Fc9AcD,,0x547d71137bd1eaad845ccd1d9b2e89edc5866861773a...,2022-11-01T18:05:35,0xBe1eDe71cF7beBe0f73Cbd59E06Ca6a6335559fF


In [140]:
streamModified_df

,block,transactionHash,logIndex,values_amountPerSec,values_from,values_newStreamId,values_oldAmountPerSec,values_oldStreamId,values_oldTo,values_timestamp,values_to
0,15018714,0xe397d5682ef780b5371f8c80670e0cd94b4f945c7b43...,354,385802469135802469,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x7afa9a156c13e95b46027efe4e6bd3e5f28ce41d520d...,707291666666666666,0xaa7b24aa90acecbae941269d7db5867eb291f5933e74...,0x8727FF875E070d73B678cCea85BF9FF01c05A785,2022-06-24T14:24:09,0x8727FF875E070d73B678cCea85BF9FF01c05A785
1,15019174,0x7bfaa362453a9320243d7f604b7ffff10c31964a62e7...,191,10912698412698412698,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x351bcbc5491a0a1119d313aa6a7e6244f3cfb5132d8a...,4133597883597883597,0x8d93da8b37ea7a3b1e92acf004cc91d8d35bea4650ab...,0x6903223578806940bd3ff0C51f87aa43968424c8,2022-06-24T16:22:36,0x6903223578806940bd3ff0C51f87aa43968424c8
2,15089008,0xeb51cb5a3b4ae618be75bf3e23c2d8e333d93d5e81e8...,189,52950767511632,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0xc7113f8b37ac9700ddaa5c46e5c245aaddd30333290a...,52950767511632,0x6f2f96966707231a42f41c71a2a4bcf543a8ae4f40f6...,0xC2De6459CAa4d11a5E3Eee3427e126BfA5FE4613,2022-07-06T12:45:37,0xBffe6Ed5795B92801366DF553c0EddE08DB5ab49
3,15089008,0xeb51cb5a3b4ae618be75bf3e23c2d8e333d93d5e81e8...,192,385802469135802432,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x9167a3d470c63a5beff8cc90b805da0abcd5fcf250a7...,385802469135802432,0x7b89987165c7432b0c67a2bfad22ef845cae1ad629d0...,0xC2De6459CAa4d11a5E3Eee3427e126BfA5FE4613,2022-07-06T12:45:37,0xBffe6Ed5795B92801366DF553c0EddE08DB5ab49
4,15166122,0x03848f4a8e206abc2ce24871be16542863845f9ea33d...,387,826719576719576720,0x6BFeBA63E725bA2610Bf251CE43De50b4d18A944,0x63545c4998fcc7b2453acda2d12c5ba442fe2b3aa92f...,826719576719576720,0x63545c4998fcc7b2453acda2d12c5ba442fe2b3aa92f...,0x292c6DAE7417B3D31d8B6e1d2EeA0258d14C4C4b,2022-07-18T10:43:09,0x292c6DAE7417B3D31d8B6e1d2EeA0258d14C4C4b
...,...,...,...,...,...,...,...,...,...,...,...
200,19059844,0x4ad2471b1d03d1bb30b77136db391a98b1aefde65bb8...,96,253678335870116692,0xc1EA2eADCD0c8a22ca3ed6d4004D21bF3037aCBA,0x1c52142d55426be1b58e42922707b897295abbea02bd...,192901234567901235,0xee593639df9852e0ba6f289eb90d3c12bda383139108...,0xe2165a834F93C39483123Ac31533780b9c679ed4,2024-01-22T04:15:23,0xe2165a834F93C39483123Ac31533780b9c679ed4
201,19093162,0x00214d2d8f44872f1ec3bbb706c9f81929b499024d4f...,481,231481481481481481,0xDdc01A9FCB5eA881E596FFD8bdE4ee66C16adc99,0xbc28ed7fc8c10c857ee9722a442686f919bbbdf43385...,135030864197530864,0x33859e2f442cee05b6e0843a265d384ec13003f2f938...,0x874d6A3E47a18Ec20387c85f4783580c635A9e62,2024-01-26T20:29:35,0x874d6A3E47a18Ec20387c85f4783580c635A9e62
202,19118601,0x61fb8362f6771952d6fce4af0c9f0b85f61ca3032d44...,128,15854895991882,0x0Cf2A3BC341038b87C80F9d9FC94B3682c1772dB,0x866cfd432df986cae65bad28ca16090370beee7df250...,3170979198376,0xd646e75ae7784d8b67fd99f52e9755cb818fedcf3dca...,0xFd048AD1C7117f3aF63b4acd8B29B0EB655f9A13,2024-01-30T10:02:47,0x657011E5848F7bD10721bFBe7F1f08144f8BC4b3
203,19118621,0x957ad791b729cce66240d02e0ba59cbe2f5a27228b4e...,64,125053906646372400,0x0Cf2A3BC341038b87C80F9d9FC94B3682c1772dB,0xa185068d994c94aa06b42c98c339a6fe0d60069be1ac...,15854895991882,0x866cfd432df986cae65bad28ca16090370beee7df250...,0x657011E5848F7bD10721bFBe7F1f08144f8BC4b3,2024-01-30T10:06:47,0xFd048AD1C7117f3aF63b4acd8B29B0EB655f9A13


In [141]:
streamPaused_df

,block,transactionHash,logIndex,values_amountPerSec,values_from,values_streamId,values_timestamp,values_to
0,15014254,0x5b2418b7cdfb94cd3af1629f6adf7a5d385231d2db75...,222,11408730158730159,0xCD298D9872823F0665b7726604dB286eD3376A8b,0x24a7a6f07d6c5b2040e5f2aed69836dd8cdd559486a3...,2022-06-23T18:28:55,0x1D84AFcf36a0dD06Bec0D17Aa961Ed9CF3066055
1,15059359,0xa287e315d16ff8448f92491a665124c58a7159fdd7cb...,598,192901234567901235,0x8a8dBE201A6688711539dC45F8033812a093b22b,0xca46a06598beb50c44e8612c4f28e4ae804776d59a11...,2022-07-01T22:52:58,0x16A13Fc7aA71f80F7d626bEA2198853b0cEe52b2
2,15059363,0x5703c0eb51786bfc5d74500240bb53009f7fee4a85ca...,280,192901234567901235,0x8a8dBE201A6688711539dC45F8033812a093b22b,0x856ed04dca047af8a703bc21e00d8cf7b685bc205533...,2022-07-01T22:54:11,0xF432A8a443922046233EDA6388B08D8bc080FFB1
3,15059367,0x446ee2d292481991be111ce881a36df9a2692e307f5d...,170,192901234567901235,0x8a8dBE201A6688711539dC45F8033812a093b22b,0xd23fd3047f300996cdc9a4bc93fed4c305cb2b8e5445...,2022-07-01T22:55:11,0xC150E9C17a9E343243AE2C22117a6FA272b1Ad91
4,15150344,0x9440431000e2950f6cfb6d6697f09ee962fbfdddd7c9...,385,82671957671957672,0x97843608a00e2bbc75ab0C1911387E002565DEDE,0x3ba5729808fb053f7992133ac9608249f6f57959acca...,2022-07-15T23:55:18,0x60583563D5879C2E59973E5718c7DE2147971807
...,...,...,...,...,...,...,...,...
128,19043388,0x37aaaed4cfe6b874ed4ab0d6a6ea03b095f38581f60f...,222,450077160493827160,0x51C2cEF9efa48e08557A361B52DB34061c025a1B,0x4b24f1d9f1b7c9cedc512c3121d23dbcee580de3b3cc...,2024-01-19T20:46:35,0xA6575f1D5Bd6545fBd34BE05259D9d6ae60641f2
129,19070077,0x0bc82f5ad087804a9c9c0f1ea9cc838deb9760a55898...,140,192901234567901235,0x3dFc49e5112005179Da613BdE5973229082dAc35,0x46ae1fa12a285cc9d337db50ef7596394fef408fdd90...,2024-01-23T14:51:23,0x60FF4545C6e674fD182990F7A66143002Fa3A03C
130,19101340,0xf7a4da9ab93fcc31d98cb048fdaabd21acb186fe1a61...,203,208950617283950617,0xc3A1BDddbeC6Bc896162F22d0D7e260Df8fE01C5,0x312207d1e40dc7498380a05f7f66214c6488370ec79e...,2024-01-28T00:01:23,0xd90E9181B20D8D1B5034d9f5737804Da182039F6
131,19140090,0x3836988a05a58713acb94835f0546a035b7e1a440578...,207,209284627092846271,0xaD6958d60Ae5b92Bf6aE770a18549EA3c703EFFf,0x9e4048480e76c0c06e39b7c1c81c25b7717434c80214...,2024-02-02T10:23:11,0x7cc349484c624F9479e6D286a7c4db9acf550604


In [143]:
withdraw_df

,block,transactionHash,logIndex,values_amount,values_amountPerSec,values_from,values_streamId,values_timestamp,values_to
0,15012918,0xcac6886f274f32c2eecb1f3e5e70adae494e0711c785...,64,11090453572916666656213,707291666666666666,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0xaa7b24aa90acecbae941269d7db5867eb291f5933e74...,2022-06-23T12:20:20,0x8727FF875E070d73B678cCea85BF9FF01c05A785
1,15013238,0xf56fa66477a2688490650f4ca5dd668d5d53b02d215f...,62,1425016534391534391229,4133597883597883597,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x8d93da8b37ea7a3b1e92acf004cc91d8d35bea4650ab...,2022-06-23T13:46:48,0x6903223578806940bd3ff0C51f87aa43968424c8
2,15013498,0xf04b43ecb2c9028b8e096f3bc877cdd907997fc97b24...,230,17986209490740740732802,1398533950617283950,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x03300bfe1a95723a315a06a37443c718b641275e8fee...,2022-06-23T14:57:40,0x45fEEBbd5Cf86dF61be8F81025E22Ae07a07cB23
3,15014254,0x5b2418b7cdfb94cd3af1629f6adf7a5d385231d2db75...,220,68999885912698414330,11408730158730159,0xCD298D9872823F0665b7726604dB286eD3376A8b,0x24a7a6f07d6c5b2040e5f2aed69836dd8cdd559486a3...,2022-06-23T18:28:55,0x1D84AFcf36a0dD06Bec0D17Aa961Ed9CF3066055
4,15015936,0x74fadbf6bb11709bd0acb3aa17899fa8dc3d45cfe769...,82,1800884589947089946704,4133597883597883597,0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52,0x8d93da8b37ea7a3b1e92acf004cc91d8d35bea4650ab...,2022-06-24T01:52:55,0x6903223578806940bd3ff0C51f87aa43968424c8
...,...,...,...,...,...,...,...,...,...
6948,19169070,0x34cf57935391b55f69e36178c9741e7114901e310992...,116,1068185185,154320987654320988,0xDdc01A9FCB5eA881E596FFD8bdE4ee66C16adc99,0xdcdc2f7ac69a153bac395548cbd1fe9992379abe5b68...,2024-02-06T12:02:11,0x35430327e9bD2E254483b2D65aFdBc9B9Ca26a26
6949,19169243,0x5f092c3e296cff8c22d940e439378cf11d968abe9373...,274,5646118721461187210282,396372399797057331,0xd8531a94100f15af7521a7B6E724aC4959E0A025,0x7d61f61f9274ae8a2a232095414ee4259afb330dbe73...,2024-02-06T12:37:11,0xAeF6ea60f6443BAD046E825C1d2b0C0B5eBC1f16
6950,19170099,0x647834263e0d2c02ed3f1160c87d6023aa05f265f019...,220,300555555,115740740740740741,0xDdc01A9FCB5eA881E596FFD8bdE4ee66C16adc99,0x3490010e3221f6e5c4fa41861dc7cb39e07b5358870a...,2024-02-06T15:31:47,0x07eE3F3F755473AC0E7A31AE0911fC68Fd32Da7b
6951,19170547,0x8a274bcc6b19ac9e2722d2c58b6c56a2d203973f3655...,197,10617052500,354166666666666667,0x79E56F5e632d4f47b07DccDeeB2cB46Faab7266D,0xa7d78b06331ceafa6e4f1c71f1917bf279ac5c987cf3...,2024-02-06T17:01:47,0x9744a5E83128708ef892D7162cdD904860A4E732
